# Amazon SageMaker Autopilot을 활용한 다이렉트 마케팅
---

---

## 목차

1. [소개](#Introduction)
1. [사전 준비사항](#Prerequisites)
1. [데이터셋 다운로드](#Downloading)
1. [Amazon S3에 데이터셋 업로드](#Uploading)
1. [SageMaker Autopilot 작업 설정](#Settingup)
1. [SageMaker Autopilot 작업 실행](#Launching)
1. [SageMaker Autopilot 작업 진행 상황 추적](#Tracking)
1. [결과](#Results)
1. [정리](#Cleanup)

## 소개

Amazon SageMaker Autopilot은 테이블 형태 데이터셋을 위한 자동화된 머신러닝(일반적으로 AutoML이라고 함) 솔루션입니다. SageMaker Autopilot은 여러 방식으로 사용할 수 있습니다: 자동 모드(이름에서 알 수 있듯이) 또는 사람의 가이드와 함께, SageMaker Studio를 통해 코드 없이, 또는 AWS SDK를 사용하여. 이 노트북은 첫 번째 예시로서 AWS SDK를 사용하여 간단히 머신러닝 모델을 생성하고 배포하는 방법을 보여줍니다.

머신러닝의 전형적인 입문 과제("Hello World" 에 해당하는)는 한 번 이상의 전화 통화 후 고객이 은행의 정기예금에 가입할지 예측하는 데이터셋을 사용하는 것입니다. 이 과제와 사용된 데이터셋에 대한 자세한 정보는 [Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing)을 참조하세요.

우편, 이메일, 전화 등을 통한 다이렉트 마케팅은 고객을 확보하는 일반적인 전술입니다. 자원과 고객의 관심이 제한적이기 때문에, 목표는 특정 제안에 참여할 가능성이 높은 잠재 고객의 하위 집합만을 대상으로 하는 것입니다. 인구통계학적 정보, 과거 상호작용, 환경적 요인과 같이 쉽게 이용 가능한 정보를 바탕으로 이러한 잠재 고객을 예측하는 것은 일반적인 머신러닝 문제입니다. 이 과제가 여러분의 조직에서 마케팅 리드 우선순위 결정으로 쉽게 적용될 수 있음을 상상할 수 있을 것입니다.

이 노트북은 이 데이터셋에서 Autopilot을 사용하여 여러 잠재적 옵션 또는 "후보"를 탐색함으로써 가장 정확한 ML 파이프라인을 얻는 방법을 보여줍니다. Autopilot에서 생성된 각 후보는 두 단계로 구성됩니다. 첫 번째 단계는 데이터셋에 대한 자동화된 특성 엔지니어링을 수행하고, 두 번째 단계는 알고리즘을 훈련하고 튜닝하여 모델을 생성합니다. 이 모델을 배포할 때도 유사한 단계를 따릅니다. 특성 엔지니어링 후 추론을 통해 리드가 추진할 가치가 있는지 결정합니다. 이 노트북에는 모델을 훈련하는 방법과 리드 집합에 대한 배치 예측을 수행하기 위해 모델을 배포하는 방법에 대한 지침이 포함되어 있습니다. 가능한 경우 Amazon SageMaker와의 상호작용을 단순화하는 고수준 SDK인 Amazon SageMaker Python SDK를 사용합니다.

다른 예제들은 다양한 방식으로 모델을 사용자 정의하는 방법을 보여줍니다. 예를 들어, 디바이스에 배포되는 모델은 일반적으로 정확도뿐만 아니라 충족해야 하는 메모리 제약이 있습니다. 다른 사용 사례에는 실시간 배포 요구사항과 지연 시간 제약이 있습니다. 지금은 간단하게 유지하겠습니다.

## 사전 준비사항

이 튜토리얼의 작업을 시작하기 전에 다음을 수행하세요:

- 훈련 및 모델 데이터에 사용할 Amazon Simple Storage Service (Amazon S3) 버킷과 접두사. 이는 Amazon SageMaker 훈련과 동일한 리전에 있어야 합니다. 아래 코드는 기본 버킷을 생성하거나, 존재하는 경우 사용합니다.
- Autopilot이 데이터에 액세스할 수 있도록 하는 IAM 역할. IAM 역할에 대한 자세한 정보는 Amazon SageMaker 문서를 참조하세요: https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam.html

In [ ]:
# cell 01
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/autopilot-dm'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

## 데이터셋 다운로드<a name="Downloading"></a>
샘플 데이터 S3 버킷에서 [다이렉트 마케팅 데이터셋](!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip)을 다운로드합니다.

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [ ]:
# cell 02
local_data_path = './bank-additional/bank-additional-full.csv'


## Amazon S3에 데이터셋 업로드<a name="Uploading"></a>

데이터셋에서 Autopilot을 실행하기 전에, 먼저 데이터셋에 명백한 오류가 없는지 확인하세요. Autopilot 프로세스는 오랜 시간이 걸릴 수 있으며, 작업을 시작하기 전에 데이터셋을 검사하는 것이 일반적으로 좋은 관행입니다. 이 특정 데이터셋은 작으므로 노트북 인스턴스 자체에서 검사할 수 있습니다. 노트북 인스턴스 메모리에 맞지 않는 더 큰 데이터셋이 있는 경우, Apache Spark와 같은 빅데이터 분석 도구를 사용하여 오프라인으로 데이터셋을 검사하세요. [Deequ](https://github.com/awslabs/deequ)는 Apache Spark 위에 구축된 라이브러리로 대용량 데이터셋에 대한 검사를 수행하는 데 도움이 될 수 있습니다. Autopilot은 최대 5GB까지의 데이터셋을 처리할 수 있습니다.


데이터를 Pandas 데이터프레임으로 읽어들이고 살펴보겠습니다.

In [ ]:
# cell 03
import pandas as pd

data = pd.read_csv(local_data_path)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

타겟 컬럼 'y'를 예측하는 데 도움이 되는 20개의 특성이 있음을 주목하세요.

Amazon SageMaker Autopilot은 데이터 전처리를 자동으로 처리해줍니다. 결측값 처리, 범주형 특성을 수치형 특성으로 변환, 데이터 스케일링, 더 복잡한 데이터 타입 처리와 같은 기존의 데이터 전처리 기법을 수행할 필요가 없습니다.

또한 데이터셋을 훈련 및 검증 분할로 나누는 것도 필요하지 않습니다. Autopilot이 이를 처리해줍니다. 하지만 테스트 세트를 분리하고 싶을 수 있습니다. 다음이 그것인데, 모델을 테스트하는 대신 마지막에 배치 추론에 사용합니다.


### 모델에서 배치 추론을 호출하기 위한 일부 데이터 예약

데이터를 훈련 및 테스트 분할로 나눕니다. 훈련 분할은 SageMaker Autopilot에서 사용됩니다. 테스트 분할은 제안된 모델을 사용하여 추론을 수행하기 위해 예약됩니다.


In [ ]:
# cell 04
train_data = data.sample(frac=0.8,random_state=200)

test_data = data.drop(train_data.index)

test_data_no_target = test_data.drop(columns=['y'])

### Amazon S3에 데이터셋 업로드
Amazon SageMaker 훈련에서 사용할 수 있도록 파일을 .csv 형식으로 Amazon Simple Storage Service (Amazon S3)에 복사합니다.

In [ ]:
# cell 05
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

## SageMaker Autopilot 작업 설정<a name="Settingup"></a>

데이터셋을 Amazon S3에 업로드한 후, Autopilot을 호출하여 이 데이터셋에서 모델을 훈련할 최적의 ML 파이프라인을 찾을 수 있습니다.

Autopilot 작업을 호출하는 데 필요한 입력은 다음과 같습니다:
* 입력 데이터셋과 모든 출력 아티팩트를 위한 Amazon S3 위치
* 예측하려는 데이터셋 컬럼의 이름 (이 경우 `y`)
* IAM 역할

현재 Autopilot은 CSV 형식의 테이블 형태 데이터셋만 지원합니다. 모든 파일이 헤더 행을 가져야 하거나, 알파벳/사전 순으로 정렬했을 때 데이터셋의 첫 번째 파일이 헤더 행을 가져야 합니다.

In [ ]:
# cell 06
auto_ml_job_input_data_config = [{
  'DataSource': {
    'S3DataSource': {
      'S3DataType': 'S3Prefix',
      'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
    }
  }
}]

output_data_config = {
  'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
}

auto_ml_problem_type_config={
  'TabularJobConfig': {
    'CompletionCriteria': {
      'MaxCandidates': 5
    },
    'ProblemType': 'BinaryClassification',
    'TargetAttributeName': 'y',
  },
}

auto_ml_job_objective = {
  "MetricName": "Accuracy"
}

데이터셋으로 해결하려는 문제 유형(`Regression, MulticlassClassification, BinaryClassification`)을 지정할 수도 있습니다. 확실하지 않은 경우, SageMaker Autopilot은 타겟 컬럼(예측하려는 컬럼)의 통계를 기반으로 문제 유형을 추론합니다.

파이프라인 평가 또는 후보의 최대 수(하나의 파이프라인 평가는 후보 모델을 생성하기 때문에 `Candidate`라고 함)를 제공하거나 전체 Autopilot 작업에 할당된 총 시간을 제공하여 SageMaker Autopilot 작업의 실행 시간을 제한할 수 있는 옵션이 있습니다. 기본 설정에서 이 작업은 실행하는 데 약 4시간이 걸립니다. 이는 Autopilot이 최적의 훈련 매개변수를 찾기 위해 사용하는 탐색 프로세스의 특성으로 인해 실행마다 다를 수 있습니다.

## SageMaker Autopilot 작업 실행<a name="Launching"></a>

이제 `create_auto_ml_job_v2` API를 호출하여 Autopilot 작업을 실행할 수 있습니다. https://docs.aws.amazon.com/cli/latest/reference/sagemaker/create-auto-ml-job-v2.html

In [ ]:
# cell 07
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-banking-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job_v2(
    AutoMLJobName=auto_ml_job_name,
    AutoMLJobInputDataConfig=auto_ml_job_input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobObjective=auto_ml_job_objective,
    AutoMLProblemTypeConfig=auto_ml_problem_type_config,
    RoleArn=role
)

## SageMaker Autopilot 작업 진행 상황 추적<a name="Tracking"></a>
SageMaker Autopilot 작업은 다음과 같은 고수준 단계로 구성됩니다:
* 데이터 분석: 데이터셋을 분석하고 Autopilot이 데이터셋에서 시도해야 할 ML 파이프라인 목록을 제시합니다. 데이터셋은 또한 훈련 및 검증 세트로 분할됩니다.
* 특성 엔지니어링: Autopilot이 데이터셋의 개별 특성과 집계 수준에서 특성 변환을 수행합니다.
* 모델 튜닝: 훈련 알고리즘(파이프라인의 마지막 단계)에 대한 최적의 하이퍼파라미터와 함께 최고 성능의 파이프라인이 선택됩니다.

In [ ]:
# cell 08
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

## 결과

이제 describe_auto_ml_job_v2 API를 사용하여 SageMaker Autopilot 작업에서 선택한 최적의 후보를 조회합니다.

In [ ]:
# cell 09
best_candidate = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

### 최적의 후보를 사용하여 배치 추론 수행

데이터셋에서 SageMaker Autopilot 작업을 성공적으로 완료했으므로, [추론 파이프라인](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html)을 사용하여 후보 중 하나에서 모델을 생성합니다.

In [ ]:
# cell 10
model_name = 'automl-banking-model-' + timestamp_suffix

model = sm.create_model(
    Containers=best_candidate['InferenceContainers'],
    ModelName=model_name,
    ExecutionRoleArn=role
)

print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))

Amazon SageMaker 배치 변환을 사용하여 배치 추론을 사용할 수 있습니다. 동일한 모델을 Amazon SageMaker 호스팅을 사용하여 온라인 추론을 수행하도록 배포할 수도 있습니다.

In [ ]:
# cell 11
transform_job_name = 'automl-banking-transform-' + timestamp_suffix

transform_input = {
    'DataSource': {
        'S3DataSource': {
            'S3DataType': 'S3Prefix',
            'S3Uri': test_data_s3_path
        }
    },
    'ContentType': 'text/csv',
    'CompressionType': 'None',
    'SplitType': 'Line'
}

transform_output = {
    'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket,prefix),
}

transform_resources = {
    'InstanceType': 'ml.m5.4xlarge',
    'InstanceCount': 1
}

sm.create_transform_job(
    TransformJobName = transform_job_name,
    ModelName = model_name,
    TransformInput = transform_input,
    TransformOutput = transform_output,
    TransformResources = transform_resources
)

변환 작업의 완료를 지켜봅니다.

In [ ]:
# cell 12
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

이제 변환 작업의 결과를 살펴보겠습니다:

In [ ]:
# cell 13
s3_output_key = '{}/inference-results/test_data.csv.out'.format(prefix);
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

data = pd.read_csv(local_inference_results_path, sep=';')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

### SageMaker Autopilot에서 탐색한 다른 후보들 보기
SageMaker Autopilot에서 탐색한 모든 후보들(다양한 하이퍼파라미터 조합을 가진 파이프라인 평가)을 보고 최종 성능 지표로 정렬할 수 있습니다.

In [ ]:
# cell 14
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
index = 1
for candidate in candidates:
  print (str(index) + "  " + candidate['CandidateName'] + "  " + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))
  index += 1

### 후보 생성 노트북
    
SageMaker AutoPilot은 후보 정의 노트북도 자동으로 생성합니다. 이 노트북은 SageMaker Autopilot이 최적의 후보에 도달하기 위해 수행한 다양한 단계를 대화형으로 단계별로 살펴보는 데 사용할 수 있습니다. 이 노트북은 병렬 처리, 사용된 하드웨어, 탐색된 알고리즘, 특성 추출 스크립트 등과 같은 다양한 런타임 매개변수를 재정의하는 데도 사용할 수 있습니다.
    
노트북은 다음 Amazon S3 위치에서 다운로드할 수 있습니다:

In [ ]:
# cell 15
sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']


### 데이터 탐색 노트북
SageMaker Autopilot은 데이터 탐색 노트북도 자동으로 생성하며, 다음 Amazon S3 위치에서 다운로드할 수 있습니다:

In [ ]:
# cell 16
sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']


## 정리

Autopilot 작업은 데이터셋 분할, 전처리 스크립트, 전처리된 데이터 등과 같은 많은 기본 아티팩트를 생성합니다. 이 코드는 주석을 해제하면 이들을 삭제합니다. 이 작업은 생성된 모든 모델과 자동 생성된 노트북도 삭제합니다.

In [ ]:
# cell 17
#s3 = boto3.resource('s3')
#bucket = s3.Bucket(bucket)

#job_outputs_prefix = '{}/output/{}'.format(prefix,auto_ml_job_name)
#bucket.objects.filter(Prefix=job_outputs_prefix).delete()